In [1]:
from IPython import embed
from dotenv import load_dotenv
from jupyter_server.auth.security import persist_config

# from langchain_llm_test import ai_message
%pip install --upgrade --quiet  docx2txt langchain-community



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -qU langchain-text-splitters



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install langchain-chroma


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -U langchain langchainhub --quiet



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader("./tax_docs/tax_with_table.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

In [6]:
len(document_list)

225

In [7]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
embedding = OpenAIEmbeddings(
    model = "text-embedding-3-large"
)

In [8]:
import getpass
import os
from dotenv import load_dotenv


from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

load_dotenv()

pinecone_api_key = os.environ.get("PAK")
os.environ["PINECONE_API_KEY"] = pinecone_api_key

pc = Pinecone(api_key=pinecone_api_key)

index_name = 'tax-table-index'
database = PineconeVectorStore.from_documents(documents=document_list, embedding=embedding, index_name=index_name)




/Users/xeroman.k/ws/inflearn-llm-app/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/xeroman.k/ws/inflearn-llm-app/.venv/lib/python3.10/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorSt

In [9]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# retrieved_docs = database.similarity_search(query, 3)

In [10]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o')


In [11]:
# ai_message = llm.invoke(prompt)

In [12]:
# ai_message.content

In [13]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [14]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [18]:
from langchain.chains import RetrievalQA
retriever = database.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [19]:
retriever.invoke(query)

[Document(id='5c66bed5-e3af-4984-a7d8-45d71acdc33f', metadata={'source': './tax_docs/tax_with_table.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 

In [16]:
ai_message = qa_chain({"query": query})

/var/folders/4b/f3tc2wns1c31fttyw185bb700000gn/T/ipykernel_22096/314547319.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


In [17]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '공제 전 연봉 5천만원에 대한 소득세를 계산하려면 기본세율이 적용됩니다. 그러나 정확한 소득세 금액을 계산하기 위해서는 세율뿐만 아니라 공제 항목들도 고려해야 합니다. 실제 최종 소득세 금액은 여러 요인에 따라 달라질 수 있습니다.'}